In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import special_ortho_group
from hisel.select import Selector, FeatureType

In [ ]:
d = 50  # total number of features to select from
n = 10000 # sample size
n_features = 8 # relevant feature for the classification

### Generate synthetic data

In [ ]:
x = np.random.uniform(size=(n, d))
features = list(np.random.choice(d, replace=False, size=n_features))
z = x[:, features]
tt = np.expand_dims(
    special_ortho_group.rvs(n_features),
    axis=0
)
zz = np.expand_dims(z, axis=2)
u = (tt @ zz)[:, :, 0]
y = np.zeros(shape=(n, 1), dtype=int)
for i in range(1, n_features):
    y += np.asarray(u[:, [i-1]] > u[:, [i]], dtype=int)

In [ ]:
selector = Selector(x, y, xfeattype=FeatureType.CONT, yfeattype=FeatureType.DISCR)

### Run selection algorithm

In [ ]:
batch_size = n // 10
minibatch_size = 200
number_of_epochs = 3
threshold = .075
device = None # run on CPU

#### Selection: We ask for the `n_features` most relevant features for the classification task

In [ ]:
selection = selector.select(
    number_of_features=n_features, 
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
    device=device,
)

#### Auto-selection: we do not prescribe the number of features to select

In [ ]:
autoselection = selector.autoselect(
    batch_size=batch_size,
    minibatch_size=minibatch_size,
    number_of_epochs=number_of_epochs,
    threshold=threshold,
    device=device,
)

In [ ]:
print(f'Expected features:\n{sorted(features)}')
print(f'Selected features:\n{sorted(selection)}')
print(f'Auto-selected features:\n{sorted(autoselection)}')

#### Visualise importance curve as more features are selected

In [ ]:
paths = selector.lasso_path()
curve = np.cumsum(np.sort(paths.iloc[-1, :])[::-1])
plt.plot(np.arange(1, 1+len(curve)), curve)

#### Visualise lasso paths of relevant features

In [ ]:
paths = selector.lasso_path()
paths.iloc[:, features].plot(figsize=(16, 10))